# clean_hh_2005

This notebook clean household level df 

## Inputs
1. ii_in.dta : household non labor income
2. ii_inr.dta : household rural income
3. ii_portad.dta : household location

Outputs
1. folio : household id
2. hh_no_savings : hh has no savings
2. hh_has_savings : hh has savings
2. hh_no_debts : hh has no debts
2. hh_has_debts : hh has debts
3. hh_has_liquid : hh has savings or debts

In [287]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os 

import sys; sys.path.append('/home/mitch/util/python')
sys.path.append('/home/mitch/school/mfl/')

from src import json_utils

In [288]:
raw = '/home/mitch/Dropbox/data/mexico_fls/raw/2005/'
interim = '/home/mitch/Dropbox/data/mexico_fls/interim2/2005/'

figs = '/home/mitch/github/mfl/fig/'
dicts= '/home/mitch/github/mfl/dicts/'

In [289]:
rename = json_utils.load_json(dicts + 'rename_hh_2005.json')
rename

{'folio': 'hhid',
 'fac_lib2': 'weight_hh',
 'ah03a': 'own_house',
 'ah03b': 'own_otherhouse',
 'ah03c': 'own_bicycles',
 'ah03d': 'own_vehicle',
 'ah03e': 'own_electronics',
 'ah03f': 'own_washmachinestove',
 'ah03g': 'own_domesticappliance',
 'ah03h': 'own_financialassets',
 'ah03i': 'own_machinary',
 'ah03j': 'own_bullcow',
 'ah03k': 'own_horsesmules',
 'ah03l': 'own_pigsgoats',
 'ah03m': 'own_poultry',
 'ah03n': 'own_otherassets',
 'ah04a_1': 'knowvalue_house',
 'ah04b_1': 'knowvalue_otherhouse',
 'ah04c_1': 'knowvalue_bicycles',
 'ah04d_1': 'knowvalue_vehicle',
 'ah04e_1': 'knowvalue_electronics',
 'ah04f_1': 'knowvalue_washmachinestove',
 'ah04g_1': 'knowvalue_domesticappliance',
 'ah04h_1': 'knowvalue_financialassets',
 'ah04i_1': 'knowvalue_machinary',
 'ah04j_1': 'knowvalue_bullcow',
 'ah04k_1': 'knowvalue_horsesmules',
 'ah04l_1': 'knowvalue_pigsgoats',
 'ah04m_1': 'knowvalue_poultry',
 'ah04n_1': 'knowvalue_otherassets',
 'ah04a_2': 'value_house',
 'ah04b_2': 'value_otherhou

In [290]:
keep = ['folio']

In [291]:
os.chdir(raw + 'book II/data/' )

crh = pd.read_stata('ii_crh.dta')
ii_in = pd.read_stata('ii_in.dta')
ii_inr = pd.read_stata('ii_inr.dta')
ii_portad = pd.read_stata('ii_portad.dta')
ii_ah = pd.read_stata('ii_ah.dta').drop(columns=['ls'])
weights = pd.read_stata('hh05w_b2.dta')

os.chdir(raw + 'book I/data/' )
i_cs = pd.read_stata('i_cs.dta')
i_cs1 = pd.read_stata('i_cs1.dta')

os.chdir(raw + 'book C/data/' )
c_cv = pd.read_stata('c_cv.dta')


In [292]:
weights.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8289 entries, 0 to 8288
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   folio     8289 non-null   object
 1   fac_lib2  8289 non-null   int32 
dtypes: int32(1), object(1)
memory usage: 161.9+ KB


In [293]:
df = (crh.merge(weights, on=['folio'], how='left')
            .merge(ii_in, on=['folio'], how='left')
            .merge(ii_inr, on=['folio'], how='left')
            .merge(ii_portad, on=['folio'], how='left')
            .merge(i_cs, on=['folio'], how='left')
            .merge(i_cs1, on=['folio'], how='left')
            .merge(ii_ah, on=['folio'], how='left')
            .merge(c_cv, on=['folio'], how='left')
            )

In [294]:
df = df.rename(columns = rename)
df = df[rename.values()]

In [295]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8134 entries, 0 to 8133
Data columns (total 56 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   hhid                               8134 non-null   object 
 1   weight_hh                          8022 non-null   float64
 2   own_house                          8133 non-null   float32
 3   own_otherhouse                     8133 non-null   float32
 4   own_bicycles                       8133 non-null   float32
 5   own_vehicle                        8133 non-null   float32
 6   own_electronics                    8132 non-null   float32
 7   own_washmachinestove               8132 non-null   float32
 8   own_domesticappliance              8131 non-null   float32
 9   own_financialassets                8132 non-null   float32
 10  own_machinary                      8132 non-null   float32
 11  own_bullcow                        8133 non-null   float

In [296]:
df.own_house.value_counts(dropna=False)

1.0    6409
3.0    1724
NaN       1
Name: own_house, dtype: int64

In [297]:
illiquid_assets = [
    'house', 
    'otherhouse',
    'bicycles',
    'vehicle',
    'electronics',
    'washmachinestove',
    'domesticappliance',
    'financialassets',
    'machinary',
    'bullcow',
    'horsesmules',
    'pigsgoats',
    'poultry',
    'otherassets'
]

In [298]:
df.own_house.value_counts()

1.0    6409
3.0    1724
Name: own_house, dtype: int64

In [299]:
for asset in illiquid_assets:
    df['own_'+asset] = df['own_'+asset].replace({1.0:1, 3.0:0}).fillna(0)
    df['knowvalue_'+asset] = df['knowvalue_'+asset].replace({1.0:1, 8.0:0})

In [300]:
for asset in illiquid_assets:
    own = df['own_' + asset] == 1
    dont_own = df['own_' + asset] == 0

    know_value = df['knowvalue_'+asset] == 1
    dk_value = df['knowvalue_'+asset] == 0

    df_own_know = df[own & know_value]

    df.loc[own & dk_value, 'value_' + asset] = (
        np.sum(df_own_know['value_' + asset] * df_own_know['weight_hh'])
        / np.sum(df_own_know['weight_hh'])
        )
    df.loc[dont_own, 'value_' + asset] = 0.0

In [301]:
df.hh_has_savings_person_not_hhm.value_counts(dropna=False)

NaN    8128
7.0       6
Name: hh_has_savings_person_not_hhm, dtype: int64

In [302]:
has_savings_types = [
    'hh_no_savings',
    'hh_has_savings_bank',
    'hh_has_savings_cooperative',
    'hh_has_savings_credit_institution',
    'hh_has_savings_batch',
    'hh_has_savings_person_not_hhm',
    'hh_has_savings_afores',
    'hh_has_savings_solidarity',
    'hh_has_savings_at_house',
    'hh_has_savings_other'
]

has_value = {
    'hh_no_savings'                       : 1.0,
    'hh_has_savings_bank'                 : 2.0,
    'hh_has_savings_cooperative'          : 3.0,
    'hh_has_savings_credit_institution'   : 4.0,
    'hh_has_savings_batch'                : 5.0,
    'hh_has_savings_person_not_hhm'       : 7.0,
    'hh_has_savings_afores'               : 8.0,
    'hh_has_savings_solidarity'           : 9.0,
    'hh_has_savings_at_house'             : 10.0,
    'hh_has_savings_other'                : 11.0
}

for savings_type in has_savings_types:
   df[savings_type] = df[savings_type].replace({
    has_value[savings_type]:1,
    np.nan:0
   })

In [303]:
"""
df['hh_no_savings']                       = df['hh_no_savings'].replace({1.0:1.0})
df['hh_has_savings_bank']                 = df['hh_has_savings_bank'].replace({2.0:1.0})
df['hh_has_savings_cooperative']          = df['hh_has_savings_cooperative'].replace({3.0:1.0})
df['hh_has_savings_credit_institution']   = df['hh_has_savings_credit_institution'].replace({4.0:1.0})
df['hh_has_savings_batch']                = df['hh_has_savings_batch'].replace({5.0:1.0})
df['hh_has_savings_person_not_hhm']       = df['hh_has_savings_person_not_hhm'].replace({7.0:1.0})
df['hh_has_savings_afores']               = df['hh_has_savings_afores'].replace({8.0:1.0})
df['hh_has_savings_solidarity']           = df['hh_has_savings_solidarity'].replace({9.0:1.0})
df['hh_has_savings_at_house']             = df['hh_has_savings_at_house'].replace({10.0:1.0})
df['hh_has_savings_other']                = df['hh_has_savings_other'].replace({11.0:1.0})
"""

"\ndf['hh_no_savings']                       = df['hh_no_savings'].replace({1.0:1.0})\ndf['hh_has_savings_bank']                 = df['hh_has_savings_bank'].replace({2.0:1.0})\ndf['hh_has_savings_cooperative']          = df['hh_has_savings_cooperative'].replace({3.0:1.0})\ndf['hh_has_savings_credit_institution']   = df['hh_has_savings_credit_institution'].replace({4.0:1.0})\ndf['hh_has_savings_batch']                = df['hh_has_savings_batch'].replace({5.0:1.0})\ndf['hh_has_savings_person_not_hhm']       = df['hh_has_savings_person_not_hhm'].replace({7.0:1.0})\ndf['hh_has_savings_afores']               = df['hh_has_savings_afores'].replace({8.0:1.0})\ndf['hh_has_savings_solidarity']           = df['hh_has_savings_solidarity'].replace({9.0:1.0})\ndf['hh_has_savings_at_house']             = df['hh_has_savings_at_house'].replace({10.0:1.0})\ndf['hh_has_savings_other']                = df['hh_has_savings_other'].replace({11.0:1.0})\n"

In [304]:
has_savings_columns = [x for x in df.columns if 'hh_has_savings' in x]
df[has_savings_columns] = df[has_savings_columns].fillna(0)
has_savings_columns

['hh_has_savings_bank',
 'hh_has_savings_cooperative',
 'hh_has_savings_credit_institution',
 'hh_has_savings_batch',
 'hh_has_savings_person_not_hhm',
 'hh_has_savings_afores',
 'hh_has_savings_solidarity',
 'hh_has_savings_at_house',
 'hh_has_savings_other']

In [305]:
df['hh_has_savings'] = df[has_savings_columns].sum(axis=1) > 0
df['hh_has_savings'].value_counts()

False    7065
True     1069
Name: hh_has_savings, dtype: int64

In [306]:
df['hh_has_debts_12mth'].value_counts(dropna=False)

2.0    5983
1.0    2054
8.0      94
NaN       3
Name: hh_has_debts_12mth, dtype: int64

In [307]:
df['hh_has_debts_12mth'] = df['hh_has_debts_12mth'].replace(
    {1.0:1.0, 2.0:0.0, 8.0:0.0}
)
has_debt    = df['hh_has_debts_12mth'] == 1.0
no_debt     = df['hh_has_debts_12mth'] == 0.0

df.loc[no_debt, 'hh_debts_12mth'] = 0.0

In [308]:
knowvalue = [x for x in df.columns if 'knowvalue' in x]
df = df.drop(columns = knowvalue)

In [309]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8134 entries, 0 to 8133
Data columns (total 43 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   hhid                               8134 non-null   object 
 1   weight_hh                          8022 non-null   float64
 2   own_house                          8134 non-null   float32
 3   own_otherhouse                     8134 non-null   float32
 4   own_bicycles                       8134 non-null   float32
 5   own_vehicle                        8134 non-null   float32
 6   own_electronics                    8134 non-null   float32
 7   own_washmachinestove               8134 non-null   float32
 8   own_domesticappliance              8134 non-null   float32
 9   own_financialassets                8134 non-null   float32
 10  own_machinary                      8134 non-null   float32
 11  own_bullcow                        8134 non-null   float

In [310]:
os.chdir(interim)
df.to_csv('hh_2005.csv', index=False)